# Step 1, 切換tensorflow版本、匯入套件

In [0]:
%tensorflow_verison 2.x

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam

# Data Preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

# Step 2,下載並整理 Fashion MNIST 資料集



In [0]:
# Load Data
(X_train, y_train0), (X_test, y_test0) = datasets.fashion_mnist.load_data()

# Reshape Size
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Normalization
X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

4423680/4422102 [==============================] - 0s 0us/step


#Step 3, 建模型
## 把第一層修改為600，之後分歧成兩個分別是450和150，最後在合起來，輸出為10


In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate, add
f_1 = Dense(600, activation='sigmoid')
f_2 = Dense(450, activation='sigmoid')
f_4 = Dense(150, activation='relu')
f_3 = Dense(600, activation='sigmoid')
f_5 = Dense(10, activation='softmax')
x = Input(shape=(784,))
h_1 = f_1(x)
h_2 = f_2(h_1)
z = f_4(h_1)
h_3 = concatenate([h_2, z])
y = f_5(h_3)

In [0]:
split_model = Model(x, y)
split_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 600)          471000      input_1[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 450)          270450      dense_5[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 150)          90150       dense_5[0][0]                    
______________________________________________________________________________________________

In [0]:
#把optimizer換成SGD, learning rate = 0.075
split_model.compile(loss='mse', 
              optimizer=SGD(lr=0.075), 
              metrics=['accuracy'])

In [0]:
#batch_size調至128, epochs = 15
split_model.fit(X_train, y_train, 
                batch_size=128, 
                epochs=15)

Epoch 1/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0347 - accuracy: 0.7594
Epoch 2/15
469/469 [==============================] - 7s 16ms/step - loss: 0.0340 - accuracy: 0.7625
Epoch 3/15
469/469 [==============================] - 7s 16ms/step - loss: 0.0334 - accuracy: 0.7676
Epoch 4/15
469/469 [==============================] - 8s 16ms/step - loss: 0.0328 - accuracy: 0.7712
Epoch 5/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0323 - accuracy: 0.7749
Epoch 6/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0318 - accuracy: 0.7788
Epoch 7/15
469/469 [==============================] - 7s 16ms/step - loss: 0.0314 - accuracy: 0.7822
Epoch 8/15
469/469 [==============================] - 7s 16ms/step - loss: 0.0310 - accuracy: 0.7868
Epoch 9/15
469/469 [==============================] - 7s 16ms/step - loss: 0.0305 - accuracy: 0.7887
Epoch 10/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0301 - accura

In [0]:
score = split_model.evaluate(X_train, y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

6/6 [==============================] - 1s 246ms/step - loss: 0.0283 - accuracy: 0.8059
Loss: 0.028265079483389854
準確率: 80.59166669845581


隱藏層中的神經元數目較範例程式減少(可訓練的參數減少)，並替換了optimizer、下修batch_size、把epoch設為15。從結果來看，神經網路的學習效果有下降，但我覺得可能因為epoch設為15讓她loss值可以下降多一點，才會讓學習的效果沒有到根源來得差很多(準確率: 87% → 80.6%)。